In [5]:
import json
import numpy as np
import cvxpy as cp

In [ ]:
with open("C:/Users/morea/OneDrive/Documents/2A/REOP/KIRO/small_1.json", 'r') as file:
    data = json.load(file)

vehicles = {}
for i in range(len(data["vehicles"])):
    vehicles[i+1] = data["vehicles"][i]["type"]

constraints = []
for i in range(len(data["constraints"])):
    constraints.append(data["constraints"][i])

lag = {}
for i in range(len(data["shops"])):
    lag[data["shops"][i]["name"]] = data["shops"][i]["resequencing_lag"]


parameters = data["parameters"]

In [ ]:
S=3
n= #nb vehicule
Rs = [ , , ] # liste des Rs


In [4]:
# Create two scalar optimization variables.
x = cp.Variable((S,n,n))
y = cp.Variable((S,n))
zs0 = cp.Variable((0,len(Rs[0]),n))
zs1 = cp.Variable((1,len(Rs[1]),n))
zs2 = cp.Variable((2,len(Rs[2]),n))
g = cp.Variable((S,n,n))

# Create two constraints.
constraints = [x + y == 1,
               x - y >= 1]

# Form objective.
obj = cp.Minimize((x - y)**2)

# Form and solve problem.
prob = cp.Problem(obj, constraints)
prob.solve()

# The optimal dual variable (Lagrange multiplier) for
# a constraint is stored in constraint.dual_value.
print("optimal (x + y == 1) dual variable", constraints[0].dual_value)
print("optimal (x - y >= 1) dual variable", constraints[1].dual_value)
print("x - y value:", (x - y).value)


optimal (x + y == 1) dual variable 0.0
optimal (x - y >= 1) dual variable 2.0
x - y value: 1.0
